# Using CausalKit to infer binary treatment with continuous and binary outcome

## Continuos outcome

### Dataset 1

In [1]:
import numpy as np
from causalkit.data import CausalDatasetGenerator, CausalData

In [2]:
# Domain setup:
# - outcome_type='continuous' for LTV
# - theta: average causal lift in LTV from receiving the feature
# - target_t_rate: expected adoption/exposure rate of the feature
# - beta_y/beta_t: how confounders influence outcome and treatment (introduces confounding)

gen = CausalDatasetGenerator(
    theta=2.5,                      # average uplift in LTV from receiving the feature
    outcome_type="continuous",
    sigma_y=5.0,                    # noise level for LTV
    target_t_rate=0.1,             # ~30% users receive the feature
    seed=123,

    # Confounders (5 total)
    confounder_specs=[
        {"name": "tenure_days",       "dist": "normal",   "mu": 120, "sd": 60},   # days since install
        {"name": "prior_30d_spend",   "dist": "uniform",  "a": 0.0,  "b": 100.0}, # prior spend in last 30 days
        {"name": "engagement_rate",   "dist": "uniform",  "a": 0.0,  "b": 1.0},   # daily active ratio
        {"name": "premium_user",      "dist": "bernoulli","p": 0.15},             # subscription flag
        {"name": "android",           "dist": "bernoulli","p": 0.55},             # OS flag
    ],

    # Linear effects of confounders
    # Order matches the confounder_specs above
    beta_y=np.array([ 0.02,  0.10,  8.0,  5.0,  0.5]),   # effect on LTV baseline
    beta_t=np.array([ 0.01,  0.02,  1.0,  0.8, -0.3]),   # effect on probability of receiving feature (log-odds)
)

# Create CausalData directly (treatment='t', outcome='y' are set internally)
# Optionally pass the confounder names to keep only a subset; here we include all.
causal_data = gen.to_causal_data(
    n=50_000,
    cofounders=["tenure_days", "prior_30d_spend", "engagement_rate", "premium_user", "android"]
)

# Example access
causal_data.df.head()

,y,t,tenure_days,prior_30d_spend,engagement_rate,premium_user,android
0,8.698079,0.0,60.652719,26.563015,0.757982,0.0,1.0
1,16.124146,1.0,97.932801,65.694021,0.487610,0.0,0.0
2,10.631496,0.0,197.275516,19.183593,0.752212,0.0,0.0
3,7.171113,0.0,131.638465,25.349005,0.144235,0.0,1.0
4,17.188792,0.0,175.213854,34.416637,0.833235,0.0,0.0


In [3]:
# 3. Run EDA - eda.py
from causalkit.eda.eda import CausalEDA
eda = CausalEDA(causal_data, n_splits=3, random_state=42)
health = eda.data_health_check()
missing = eda.missingness_report()
summary = eda.summaries()
ps = eda.fit_propensity()
auc = eda.treatment_predictability_auc(ps)
positivity = eda.positivity_check(ps)
balance = eda.balance_table(ps, estimand='ATE')
weights = eda.weight_diagnostics(ps, estimand='ATE')

print('Health:', health)
print('Treatment rate:', summary['treatment_rate'])
print('AUC:', auc)
print('Positivity flag:', positivity['flag'])
print('Balance (head):')
print(balance.head())



Health: {'constant_columns': [], 'n_duplicates': 0, 'n_rows': 50000}
Treatment rate: 0.09978
AUC: 0.7475463816045251
Positivity flag: True
Balance (head):
         covariate  SMD_unweighted  SMD_weighted  flag_unw  flag_w
0      tenure_days        0.586909     -0.005641      True   False
1  prior_30d_spend        0.542104     -0.016668      True   False
2  engagement_rate        0.296096      0.015714      True   False
3     premium_user        0.291716     -0.004577      True   False
4          android       -0.168447      0.014151      True   False


In [4]:
summary

{'treatment_rate': np.float64(0.09978),
 'outcome_by_treatment':      count       mean       std
 t                              
 0.0  45011  12.103740  6.525400
 1.0   4989  18.116251  6.457517,
 'naive_diff': np.float64(6.01251122740617)}

In [5]:
positivity

{'bounds': (0.05, 0.95),
 'share_below': 0.34994,
 'share_above': 0.0,
 'flag': True}

In [6]:
# 4. Run inference - inference (DoubleML)
from causalkit.inference.att import dml_att

# ATT (optional)
res_att = dml_att(causal_data, n_folds=3, n_rep=1, confidence_level=0.95)
print('ATT:', res_att['coefficient'], 'CI:', res_att['confidence_interval'])


ATT: 2.656014559833901 CI: (np.float64(2.4954436274298453), np.float64(2.8165854922379565))


In [7]:
from causalkit.inference.ate import dml_ate
res_ate = dml_ate(causal_data)
print('ATE', res_ate['coefficient'], 'CI:', res_att['confidence_interval'])

ATE 2.5741252410065254 CI: (np.float64(2.4954436274298453), np.float64(2.8165854922379565))


In [8]:
res_att

{'coefficient': np.float64(2.656014559833901),
 'std_error': np.float64(0.08192545050348818),
 'p_value': np.float64(1.439419954878669e-230),
 'confidence_interval': (np.float64(2.4954436274298453),
  np.float64(2.8165854922379565)),
 'model': <doubleml.irm.irm.DoubleMLIRM at 0x15f753620>}

In [9]:
from causalkit.refutation import refute_placebo_outcome, refute_placebo_treatment, refute_subset

In [10]:
att_placebo_treatment = refute_placebo_treatment(
    dml_att,
    causal_data,
    random_state=42
)

print(f"\nATT with shuffled treatment:")
print(f"  θ = {att_placebo_treatment['theta']:.4f}")
print(f"  p-value = {att_placebo_treatment['p_value']:.4f}")


ATT with shuffled treatment:
  θ = -0.0218
  p-value = 0.7758


In [11]:
# Test with ATT estimator
att_placebo_outcome = refute_placebo_outcome(
    dml_att,
    causal_data,
    random_state=42
)

print(f"\nATT with shuffled outcome:")
print(f"  θ = {att_placebo_outcome['theta']:.4f}")
print(f"  p-value = {att_placebo_outcome['p_value']:.4f}")



ATT with shuffled outcome:
  θ = 0.1349
  p-value = 0.2364


In [13]:
subset_fractions = [0.6, 0.8, 0.9]

att_subset_results = []
for fraction in subset_fractions:
    subset_result = refute_subset(
        dml_att,
        causal_data,
        fraction=fraction,
        random_state=42
    )
    att_subset_results.append(subset_result)

    print(f"  {fraction*100:.0f}% subset: θ = {subset_result['theta']:.4f}, p = {subset_result['p_value']:.4f}")

  60% subset: θ = 2.6818, p = 0.0000
  80% subset: θ = 2.6662, p = 0.0000
  90% subset: θ = 2.6790, p = 0.0000
